In [5]:
import os
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.stats import multivariate_normal as MVN
from scipy.optimize import minimize_scalar
from plotnine import *

In [6]:
print(2+2)

4


## (1) Background

Consider the sum of a normal $Z_1 \sim N(\mu_1,\tau_1^2)$ and truncated normal variable and $Z_2 \sim TN(\mu_2,\tau_2^2,a,b)$: $W=Z_1+Z_2$. How can this normal-truncated sum (NTS) distribution be characterized? One approach for doing inference on $W$ is simply to sample from it, as sampling algorithms for both $Z_1$ and $Z_2$ are well defind. Having an analytical formula for the density (PDF) and cumulative distribution function (CDF) for $W$ allows for an exact calculations which can enable faster inference over a range of different parameters and removes the randomness inherant in sampling. This post will show how to 1) calculate the PDF and CDF for $W$, and 2) provide three statistical use cases: i) quality control, ii) two-stage hypothesis testing, and iii) data carving for selective inference. 

This post will use theoretical results from [Arnold et. al (1993)](https://link.springer.com/article/10.1007/BF02294652) (hereafter Arnold) and [Kim (2006)](https://www.kss.or.kr/jounalDown.php?IDX=831) (hereafter Kim), and show how to implement their work in `python`.  

## (2) Integrating a bivariate normal CDF

The CDF of a bivarate normal will turn out to be necessary to calculate the CDF of the NTS. While there is no closed-form solution to the CDF of a BVN (unless it's to calculate the the CDF from the origin), the integration problem can be written fairly succinctly and solved by fast integration solvers. Furthermore, there are analytical solutions an approximation of the CDF. This section will briefly review both approaches. 



## (2) Deriving $f_W$ and $f_W$

The first step to characterize the distribution NTS, $W$, is to understand the distribution of a truncated bivariate normal distrubion (TBVN). 

$$
\begin{align*}
(X, Y) &\sim TBVN(\theta, \Sigma, \rho, a, b) \\
E([X,Y]) &= [\theta_1, \theta_2] \\
V([X,Y]) &= \begin{pmatrix} \sigma_1^2 & \rho \sigma_1\sigma_2 \\ \rho \sigma_1\sigma_2 & \sigma_2^2  \end{pmatrix} \\
X &\in \mathbb{R}, \hspace{3mm} Y \in [a,b]
\end{align*}
$$

Notice that the TBVN takes the same formulation as a bivariate normal (a mean vector, a covariance matrix, and a correlation coefficient $\rho$) except that the random variable $Y$ term is truncated bound between $a$ and $b$. Arnold showed that the marginal density of the non-truncated random variabel ($X$) could be written as follows:

$$
\begin{align}
f_X(x) &= \frac{\phi(m_1(x)) \Bigg[ \Phi\Big(\frac{\beta-\rho \cdot m_1(x)}{\sqrt{1-\rho^2}}\Big) - \Phi\Big(\frac{\alpha-\rho \cdot m_1(x)}{\sqrt{1-\rho^2}}\Big) \Bigg] }{\sigma_1\cdot Z} \tag{1} \\ 
m_i(x) &= (x - \theta_i)/\sigma_i  \\
\alpha &= \frac{a-\theta_2}{\sigma_2}, \hspace{3mm} \beta = \frac{b-\theta_2}{\sigma_2}  \\
Z &= \Phi(\beta) - \Phi(\alpha)
\end{align}
$$

Kim showed that a NTS could be written as a TBVN by a simple change of variables:

$$
\begin{align*}
X &= Z_1 + Z_2^u \\ 
Y &= Z_2^u, \hspace{3mm} Y \in [a,b] \\
Z_2^u &\sim N(\mu_2,\tau_2^2),
\end{align*}
$$

Where $Z_2^u$ is the non-truncated version of $Z_2$. Clearly this specification of the TBVN is equivalent to the NTS and the marginal distribution of $X$ is equivalent to the PDF of $W$. 

$$
\begin{align*}
F_X(x) &= 1 - \frac{L(m_1(x),\alpha,\rho) - L(m_1(x),\beta,\rho)}{Z}
\end{align*}
$$

Where $L(d,e,f)=P(X_1 > d, X_2>e; f)$ is the orthant probability of the standard bivariate normal distribution (i.e. $\mu=(0,0)$, $\Sigma=((1,f),(f,1)))$, $|f|\leq 1$).

